# 0.0-JV-WebData
The code below will prepare the weekday average .csv to be visualized on the project website.

In [1]:
import datetime
import pandas as pd

def groupby_min(df, dur):
    """Using the hadive data, this function with take a dataframe and 
    groupby an average time interval (e.g., 5 min).
    Arguments:
    df - df to groupby.
    dur - time duration to groupby."""
    temp = df.copy()
    temp.index = map(lambda x: pd.datetime.combine(datetime.datetime.today(), x), temp.index.time)
    return temp.groupby("cam_id").resample(dur).mean()

df = pd.read_csv("../data/external/hadive-data.csv",
                 usecols=["cam_id", "date", "cam_dir", "count", "imtime"],
                 dtype={"cam_id": int, "date": str, "cam_dir": str, "count": int, "imtime": str},
                 parse_dates=["date"], infer_datetime_format=True, index_col="date")
df = df[df.index.weekday < 5] # Select weekdays.

dfs = []
for cam_id in df.cam_id.unique(): # Get 5 min averages for all cameras.
    dfs.append(groupby_min(df[df["cam_id"] == cam_id], "5Min").set_index("cam_id", append=True))

df_ = pd.concat(dfs) # Concatenate all 5 min averages.
df_ = df_.reset_index(level=2, drop=True).reset_index().fillna(0)
df_.columns = ["cam", "date", "count"]
df_.set_index("cam").to_csv("..data/external/weekdayavg.csv")